<a href="https://colab.research.google.com/github/romicabrera/Data-science/blob/main/Transacciones_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession

import sys
import os
os.environ["PYSPARK_PYTHON"] = sys.executable

# Crear sesión de Spark
spark = SparkSession.builder.appName("AnalisisTransacciones").getOrCreate()


# 2. Cargar el archivo CSV
file_path = 'transacciones2.csv'
df_raw = spark.read.csv(file_path, header=True, inferSchema=True)
rdd = df_raw.rdd


# Ver las primeras líneas
rdd.take(5)

header = rdd.first()
rdd = rdd.filter(lambda x: x != header)

rdd_mapeado = rdd.map(lambda x: (x[1], float(x[2]), x[4]))

rdd_mapeado.take(5)

rdd_filtrado = rdd_mapeado.filter(lambda x: x[1] > 1000)
rdd_filtrado.collect()

rdd_gastos = rdd_mapeado.map(lambda x: (x[0], x[1])) \
.reduceByKey(lambda x, y: x + y)

rdd_gastos.collect()

montos = rdd_mapeado.map(lambda x: x[1])
total = montos.sum()
promedio = montos.mean()
desviacion = montos.stdev()

print(f"Total: {total}, Promedio: {promedio}, Desviación estandar: {desviacion}")

top_usuarios = rdd_gastos.sortBy(lambda x: x[1], ascending=False).take(3)
print(top_usuarios)

Total: 9800.0, Promedio: 1400.0, Desviación estandar: 696.9320524371695
[('Carla', 3800.0), ('Ana', 3300.0), ('Pedro', 2200.0)]
